In [11]:
import pandas as pd

# Caminho do arquivo
caminho = 'dados/transacoes_simuladas.csv'

# Ler o CSV (usando o mesmo separador que usamos ao criar — ";")
df = pd.read_csv(caminho, sep=';')

# Exibir as 5 primeiras linhas
print(df.head())

# Verificar informações gerais do DataFrame
print("\nInformações do DataFrame:")
print(df.info())

      tipo               descricao    valor        data    categoria
0  despesa   Despesa referente a 4  6989.67  2025-01-14      compras
1  despesa   Despesa referente a 9  3897.88  2025-03-31   transporte
2  despesa   Despesa referente a 4  8920.05  2025-02-23  alimentação
3  receita  Receita referente a 10  2039.33  2025-08-30      aluguel
4  despesa   Despesa referente a 8  4617.15  2025-07-04     educação

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tipo       100 non-null    object 
 1   descricao  100 non-null    object 
 2   valor      100 non-null    float64
 3   data       100 non-null    object 
 4   categoria  100 non-null    object 
dtypes: float64(1), object(4)
memory usage: 4.0+ KB
None


In [ ]:
# Total de receitas e despesas
total_receitas = df.loc[df["tipo"] == "receita", "valor"].sum()
total_despesas = df.loc[df["tipo"] == "despesa", "valor"].sum()
saldo_atual = total_receitas - total_despesas

print(f"\n💰 Total de Receitas: R$ {total_receitas:.2f}")
print(f"💸 Total de Despesas: R$ {total_despesas:.2f}")
print(f"📊 Saldo Atual: R$ {saldo_atual:.2f}")

# Gastos por categoria
gastos_categoria = (
    df[df["tipo"] == "despesa"]
    .groupby("categoria")["valor"]
    .sum()
    .sort_values(ascending=False)
)
print("\n🔹 Gastos por categoria:")
print(gastos_categoria)

# Saldo mensal
df["ano_mes"] = df["data"].dt.to_period("M")
saldo_mensal = (
    df.groupby(["ano_mes", "tipo"])["valor"]
    .sum()
    .unstack(fill_value=0)
)
saldo_mensal["saldo"] = saldo_mensal["receita"] - saldo_mensal["despesa"]

print("\n📆 Saldo Mensal:")
print(saldo_mensal)

Importado CSV → TXT em dados/transacoes.txt
